In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pickle

In [3]:
Returns = pd.read_csv('Returns.csv')

In [4]:
Returns

,Date,MMM,MSFT,JPM,PLD,AMZN,XOM,JNJ,ATT,1Month
0,2019-07-15,0.007937,0.034548,0.023604,0.016968,-0.010929,-0.002400,0.003223,0.041476,2.17
1,2019-07-22,-0.019715,-0.031414,-0.028308,-0.019486,-0.061661,-0.040904,0.002601,0.000586,2.13
2,2019-07-29,-0.041513,0.005917,-0.028248,0.016396,-0.008589,-0.012683,0.007400,0.010828,2.12
3,2019-08-05,-0.012051,-0.011473,-0.018407,0.006844,-0.008304,-0.035855,-0.005150,0.012450,2.07
4,2019-08-12,-0.026154,-0.016865,-0.015782,-0.006069,-0.023960,0.000427,-0.027634,-0.004290,2.09
...,...,...,...,...,...,...,...,...,...,...
256,2024-06-10,0.014767,0.016291,0.013004,-0.025737,0.029511,0.015122,0.022056,0.043084,5.47
257,2024-06-17,-0.001953,-0.006292,0.030362,0.035452,0.022054,0.039364,-0.017412,0.038587,5.45
258,2024-06-24,-0.008514,0.046113,0.012509,0.019143,0.034929,-0.015202,0.002189,-0.017792,5.42
259,2024-07-01,0.026846,-0.029964,0.006278,0.049712,-0.027550,-0.000882,0.023211,0.002131,5.48


In [5]:
n_runs = 20
n_stock = 5
seq_len = 4
n_iter = 2000
batch = 10

outRtn_dates = []

for scene in range(n_runs):
    scene = scene*10
    out_of_sample_idx = scene + batch + seq_len
    outRtn_dates.append(Returns.iloc[out_of_sample_idx]['Date'])

In [6]:
outRtn_dates

['2019-10-21',
 '2019-12-30',
 '2020-03-09',
 '2020-05-18',
 '2020-07-27',
 '2020-10-05',
 '2020-12-14',
 '2021-02-22',
 '2021-05-03',
 '2021-07-12',
 '2021-09-20',
 '2021-11-29',
 '2022-02-07',
 '2022-04-18',
 '2022-06-27',
 '2022-09-05',
 '2022-11-14',
 '2023-01-23',
 '2023-04-03',
 '2023-06-12']

In [7]:
out_rates = Returns[Returns['Date'].isin(outRtn_dates)]['1Month'].values

In [8]:
out_rates

array([1.76, 1.51, 0.57, 0.1 , 0.1 , 0.09, 0.07, 0.03, 0.02, 0.05, 0.06,
       0.07, 0.03, 0.38, 1.16, 2.45, 3.72, 4.69, 4.7 , 5.24])

## SCOT Method

In [9]:
SCOT_Results = np.zeros((9, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        logdir = './logs/mv_SCOT_radi_{}_risk_{}/DRORtn.pickle'.format(radi, risk_aver)
        with open(logdir, 'rb') as fp:
            SCOT_Rtn = pickle.load(fp)
        SCOT_Results[idx, :] = SCOT_Rtn
        idx += 1
        print('Radius', radi, 'Risk aversion', risk_aver, SCOT_Rtn.mean()/SCOT_Rtn.std())
        print(SCOT_Rtn.mean(), SCOT_Rtn.std())

Radius 0.05 Risk aversion 0.0 18.333828071532213
1.0218985243822711 0.05573841537049321
Radius 0.1 Risk aversion 0.0 18.429191431378026
1.0225107773604774 0.055483214289017785
Radius 0.2 Risk aversion 0.0 18.174804727825126
1.0202998485743182 0.056138146398473664
Radius 0.05 Risk aversion 0.1 21.600879963901285
1.0362651158433418 0.04797328245771078
Radius 0.1 Risk aversion 0.1 21.107104745791755
1.0305976641712515 0.048827050255517764
Radius 0.2 Risk aversion 0.1 18.18643650036659
1.0115387938598484 0.05562050563558498
Radius 0.05 Risk aversion 1.0 20.729889906443155
1.0273922358771064 0.04956091134655654
Radius 0.1 Risk aversion 1.0 19.278927124954155
1.0188521673308333 0.05284797025930228
Radius 0.2 Risk aversion 1.0 17.846970458130766
1.0170872929013546 0.05698935263480461


In [10]:
SCOT_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        SCOT_df.loc[idx, 'RiskAversion'] = risk_aver
        SCOT_df.loc[idx, 'Radius'] = radi
        ExcessReturn = SCOT_Results[idx, :] - out_rates/12*0.01 - 1
        SCOT_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
        SCOT_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
        SCOT_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
        idx += 1

In [11]:
SCOT_df

,RiskAversion,Radius,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.05,0.020782,0.056115,1.282913
1,0.0,0.10,0.021394,0.055855,1.326864
2,0.0,0.20,0.019183,0.056554,1.175029
3,0.1,0.05,0.035148,0.048384,2.516465
4,0.1,0.10,0.029481,0.049208,2.075379
5,0.1,0.20,0.010422,0.055984,0.644884
6,1.0,0.05,0.026276,0.049918,1.823429
7,1.0,0.10,0.017736,0.053126,1.156442
8,1.0,0.20,0.015971,0.057317,0.965234


In [12]:
SCOT_df.to_csv('SCOT_df.csv', header=True, index=False)

## Non-robust method

In [13]:
Non_Results = np.zeros((3, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    logdir = './logs/mv_Nonrobust_risk_{}/DRORtn.pickle'.format(risk_aver)
    with open(logdir, 'rb') as fp:
        Non_Rtn = pickle.load(fp)
    Non_Results[idx, :] = Non_Rtn
    idx += 1
    print('Risk aversion', risk_aver, Non_Rtn.mean()/Non_Rtn.std())
    print(Non_Rtn.mean(), Non_Rtn.std())

Risk aversion 0.0 16.47121294713709
1.0209980991982672 0.061986819214533306
Risk aversion 0.1 16.89390594292983
1.0253407014950717 0.06069293299955781
Risk aversion 1.0 17.684888723819785
1.0235186634700792 0.05787532392508082


In [14]:
Non_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    Non_df.loc[idx, 'RiskAversion'] = risk_aver
    ExcessReturn = Non_Results[idx, :] - out_rates/12*0.01 - 1
    Non_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
    Non_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
    Non_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
    idx += 1

In [15]:
Non_df

,RiskAversion,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.019881,0.062420,1.103347
1,0.1,0.024224,0.061018,1.375235
2,1.0,0.022402,0.058163,1.334227


In [16]:
Non_df.to_csv('Non_df.csv', header=True, index=False)

## Naive method (As a sanity check, it should be the same while risk aversion/radius is different)

In [26]:
Naive_Results = np.zeros((3, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    logdir = './logs/mv_Nonrobust_risk_{}/NaiveRtn.pickle'.format(risk_aver)
    with open(logdir, 'rb') as fp:
        Naive_Rtn = pickle.load(fp)
    Naive_Results[idx, :] = Naive_Rtn
    idx += 1
    print('Risk aversion', risk_aver, Naive_Rtn.mean()/Naive_Rtn.std())
    print(Naive_Rtn.mean(), Naive_Rtn.std())

Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972


In [18]:
Naive_Results = np.zeros((9, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        logdir = './logs/mv_SCOT_radi_{}_risk_{}/NaiveRtn.pickle'.format(radi, risk_aver)
        with open(logdir, 'rb') as fp:
            Naive_Rtn = pickle.load(fp)
        Naive_Results[idx, :] = Naive_Rtn
        idx += 1
        print('Radius', radi, 'Risk aversion', risk_aver, Naive_Rtn.mean()/Naive_Rtn.std())
        print(Naive_Rtn.mean(), Naive_Rtn.std())

Radius 0.05 Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.1 Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.2 Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.05 Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.1 Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.2 Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.05 Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.1 Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.2 Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972


In [19]:
Naive_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    Naive_df.loc[idx, 'RiskAversion'] = risk_aver
    ExcessReturn = Naive_Results[idx, :] - out_rates/12*0.01 - 1
    Naive_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
    Naive_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
    Naive_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
    idx += 1

In [20]:
Naive_df

,RiskAversion,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.009959,0.052015,0.663254
1,0.1,0.009959,0.052015,0.663254
2,1.0,0.009959,0.052015,0.663254


In [21]:
Naive_df.to_csv('Naive_df.csv', header=True, index=False)

## OT Method

In [22]:
OT_Results = np.zeros((9, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        logdir = './logs/mv_OT_radi_{}_risk_{}/DRORtn.pickle'.format(radi, risk_aver)
        with open(logdir, 'rb') as fp:
            OT_Rtn = pickle.load(fp)
        OT_Results[idx, :] = OT_Rtn
        idx += 1
        print('Radius', radi, 'Risk aversion', risk_aver, OT_Rtn.mean()/OT_Rtn.std())
        print(OT_Rtn.mean(), OT_Rtn.std())

Radius 0.05 Risk aversion 0.0 19.444214813468562
1.011318461272276 0.052011277954600615
Radius 0.1 Risk aversion 0.0 18.87868219814078
1.0127894715060966 0.05364725465879385
Radius 0.2 Risk aversion 0.0 19.654746371978852
1.0188076094439746 0.051835194927595515
Radius 0.05 Risk aversion 0.1 19.928738899124102
1.0132159139999874 0.05084194836054176
Radius 0.1 Risk aversion 0.1 19.1814569160587
1.0130609592839426 0.052814599209917615
Radius 0.2 Risk aversion 0.1 20.834881023566513
1.0216407256868827 0.049035112057097714
Radius 0.05 Risk aversion 1.0 19.650365471254855
1.01105694010774 0.051452322430681735
Radius 0.1 Risk aversion 1.0 19.449945527717826
1.011221589782684 0.05199097284573919
Radius 0.2 Risk aversion 1.0 19.55179067472692
1.0127259463491458 0.05179709435301074


In [23]:
OT_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        OT_df.loc[idx, 'RiskAversion'] = risk_aver
        OT_df.loc[idx, 'Radius'] = radi
        ExcessReturn = OT_Results[idx, :] - out_rates/12*0.01 - 1
        OT_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
        OT_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
        OT_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
        idx += 1

In [24]:
OT_df

,RiskAversion,Radius,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.05,0.010202,0.052347,0.675112
1,0.0,0.10,0.011673,0.054013,0.748633
2,0.0,0.20,0.017691,0.052266,1.172529
3,0.1,0.05,0.012099,0.051181,0.818912
4,0.1,0.10,0.011944,0.053193,0.777849
5,0.1,0.20,0.020524,0.049505,1.436157
6,1.0,0.05,0.009940,0.051777,0.665048
7,1.0,0.10,0.010105,0.052340,0.668790
8,1.0,0.20,0.011609,0.052175,0.770787


In [25]:
OT_df.to_csv('OT_df.csv', header=True, index=False)

In [ ]:
import torch

In [ ]:
x=torch.ones(2, requires_grad=True)
y=1.2*x
z=3+x.detach()
r=(y+z).sum()

r.backward()
x.grad

In [ ]:
z.grad